In [ ]:
import os
from glob import glob
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
outputs_root = "C:/JSPRuns/MultiRuns/Outputs"

dat_files = glob(outputs_root + "/**/*.dat", recursive=True)
dat_files

In [ ]:
dat_file = dat_files[0]

In [ ]:
with open(dat_file) as f:
    for line in f:
        print(line)

In [ ]:
df = pd.read_csv(dat_file,skipfooter=1,delimiter="\t")
df = df.loc[:,~df.columns.str.match(r"^Unnamed")]
df

In [ ]:
dfs = []
for dat_file in dat_files:
    df = pd.read_csv(dat_file,skipfooter=1,delimiter="\t")
    df = df.loc[:,~df.columns.str.match(r"^Unnamed")]
    df.loc[:,"run"] = os.path.splitext(os.path.split(dat_file)[-1])[0]
    dfs.append(df)


In [ ]:
df = pd.concat(dfs).reset_index(drop=True)


In [ ]:
def date_or_nan(x):
    try:
        return datetime.strptime(x[0],"%Y-%m-%d-%H%M%S")
    except ValueError:
        return pd.NaT
        
df["run_date"] = df["run"].str.split("_").apply(date_or_nan)
df["run_id"] = df["run"].str.extract(r"_(\d{4})_")
df
 

In [ ]:
grp_col = "run_id"
val_col = "[cvMWUnloadFL].TotalIn([Ore])/1[Mt]"

box_plot = sns.boxplot(data=df, x=grp_col,y=val_col,)

means = df.groupby(grp_col)[val_col].mean()
vertical_offset = df[val_col].mean() * 0.01 # offset from median for display

for xtick in box_plot.get_xticks():
    box_plot.text(xtick,means[xtick] + vertical_offset,round(means[xtick],ndigits=1), 
            horizontalalignment='center',size='x-small',color='k',weight='semibold')

plt.grid(True)

In [ ]:
col = "[cvMWUnloadFL].TotalIn([Ore])/1[Mt]"
df.groupby("run")[col].mean().sort_values().round(decimals=1)